<a href="https://colab.research.google.com/github/abdulwaqar844/BATCH62_Q2_Assignment_01/blob/main/RAG_WITH_PDF_FILE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain  langchain_pinecone pinecone google-generativeai openai tqdm chromadb langchain_community langchain-google-genai pypdf PyPDF2 --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 1.3 MB/s eta 0:0

In [3]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
PINECONE_API_KEY=userdata.get('PINECONE_API_KEY')

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

chat_model = ChatGoogleGenerativeAI(google_api_key=GOOGLE_API_KEY,model="gemini-1.5-flash")

In [5]:

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

# Load the PDF using PyPDFLoader
loader = PyPDFLoader("/content/contract.pdf")
documents = loader.load_and_split()  # Returns a list of Document objects

In [6]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
# embedding_model = GoogleGenerativeAIEmbeddings(google_api_key=GOOGLE_API_KEY, model="models/embedding-001")
embedding_model = GoogleGenerativeAIEmbeddings(google_api_key=GOOGLE_API_KEY,  model="models/text-embedding-004",  chunk_size=100,task_type="retrieval_document")


In [7]:
from pinecone import Pinecone, ServerlessSpec
import time

pinecone_api_key=userdata.get('PINECONE_API_KEY')

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "langchain-rag-pdf"  # change if desired
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)


In [11]:
from langchain_pinecone import PineconeVectorStore
embeddings = GoogleGenerativeAIEmbeddings(google_api_key=GOOGLE_API_KEY,  model="models/text-embedding-004",  chunk_size=500,task_type="retrieval_document")
vector_store = PineconeVectorStore(index=index, embedding=embeddings)


In [10]:
llm = ChatGoogleGenerativeAI(google_api_key=GOOGLE_API_KEY,model="gemini-1.5-flash")

In [12]:
# Initialize the CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
pages = [doc.page_content for doc in documents]

# Split the text into chunks
chunks = text_splitter.create_documents(pages)
vector_store.add_documents(chunks)
# Display results
print(f"Number of chunks: {len(chunks)}")
print(f"Type of first chunk: {type(chunks[0])}")

Number of chunks: 3
Type of first chunk: <class 'langchain_core.documents.base.Document'>


In [13]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})


In [14]:

from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
chat_template = ChatPromptTemplate.from_messages([
    # System Message Prompt Template
    SystemMessage(content="""You are a Helpful AI Bot.
                  Given a context and question from user,
                  you should answer based on the given context."""),
    # Human Message Prompt Template
    HumanMessagePromptTemplate.from_template("""Answer the question based on the given context.
    Context: {context}
    Question: {question}
    Answer: """)
])

In [15]:
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs , "question": RunnablePassthrough()}
    | chat_template
    | chat_model
    | output_parser
)

In [16]:
response = rag_chain.invoke("""what will be duration of """)

response

'The Supplier will receive a monthly salary of $1200.'

In [17]:
response = rag_chain.invoke("""what will be duration of contact""")

response

'The contract duration is 12 months.'

In [21]:
response = rag_chain.invoke("""sick leaves""")

response

'Regarding absences, sick leaves, or any wage-related clause, please consult the Handbook.'